# Implementation of a CA for generating realistic clouds
Source
Application of cellular automata approach for cloud simulation and rendering
Chaos 24, 013125 (2014); https://doi.org/10.1063/1.4866854
W. Christopher Immanuel and S. Paul Mary Deborrah and R. Samuel Selvaraj

In [1]:
import torch
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
%matplotlib widget
# Parameters
# size
width = 100
depth = 100
height = 100
# init cloud position -> area between is initialised for cloud creation, the other cells for extincion
# elliptic volume, defined by center and strech factor
# width
c_x = 50
f_x = 10
# depth
c_y = 50
f_y = 10
# height
c_z = 50
f_z = 50

radius = 1. # default distance for ellipsoid calculation
overlap = 1. # overlapping of creation and extinction zone (x-c_x)**2/f_x + (y-c_y)**2/f_y + (z-c_z)**2/f_z <= radius + overlap

# Probabilities for creating and extinction: int16 [0...10000] = 0.00% ... 100.00%
P_hum0 = 100
P_act0 = 1
P_ext0 = 5000

# iterations for CA simulation
n_iteration = 40

# init variable grid as tensors
dev = torch.device("cuda")

hum = torch.zeros((width, depth, height), device=dev, dtype=torch.uint8) # humidity/vapor
act = torch.zeros((width, depth, height), device=dev, dtype=torch.uint8) # activation/phase transition factor
cld = torch.zeros((width, depth, height), device=dev, dtype=torch.uint8) # clouds
f_act = torch.zeros_like(act) # activation factor -> helper variable for calculation
hum_temp = torch.zeros_like(hum) # temporary tensor for humidity, since it is also used for act calculation

In [3]:
# Variables for creation and extinction process
# reserve memory for random number creation
rnd_hum = torch.zeros_like(hum, dtype=torch.int16)
rnd_act = torch.zeros_like(act, dtype=torch.int16)
rnd_ext = torch.zeros_like(cld, dtype=torch.int16)

# init probability areas for random variable changes
P_hum = torch.zeros_like(hum, dtype=torch.int16)
P_act = torch.zeros_like(act, dtype=torch.int16)
P_ext = torch.zeros_like(cld, dtype=torch.int16)

# get all coordinates
x, y, z = torch.meshgrid([torch.arange(0,width), torch.arange(0,depth), torch.arange(0,height)])

# create selection masks
distance = (x-c_x)**2/f_x + (y-c_y)**2/f_y + (z-c_z)**2/f_z
# inner
sel_inner = distance <= radius + overlap
# outer
sel_outer = distance > radius - overlap

P_hum = P_hum0 # humidity for complete volume
P_act[sel_inner] = P_act0
P_ext[sel_outer] = P_ext0


In [12]:
# prepare all cloud coordinates
x = x.reshape(1, -1).squeeze()
y = y.reshape(1, -1).squeeze()
z = z.reshape(1, -1).squeeze()
# prepare visualisation
fig = plt.figure()
ax3D = fig.add_subplot(111, projection='3d')

FigureCanvasNbAgg()

In [13]:
for i in range(n_iteration):
    # calculate new hum
    hum_temp = hum & ~act
    # calculate new cld
    cld = cld | act
    # calculate new activation factor
    f_act = (
             torch.cat((act[-2:,:,:], act[:-2,:,:]), dim=0) | torch.cat((act[-1:,:,:], act[:-1,:,:]), dim=0) | 
             torch.cat((act[1:,:,:], act[:1,:,:]), dim=0) | torch.cat((act[2:,:,:], act[:2,:,:]), dim=0) |
             torch.cat((act[:,-2:,:], act[:,:-2,:]), dim=1) | torch.cat((act[:,-1:,:], act[:,:-1,:]), dim=1) | 
             torch.cat((act[:,1:,:], act[:,:1,:]), dim=1) |
             torch.cat((act[:,:,-2:], act[:,:,:-2]), dim=2) | torch.cat((act[:,:,-1:], act[:,:,:-1]), dim=2) | 
             torch.cat((act[:,:,1:], act[:,:,:1]), dim=2) | torch.cat((act[:,:,2:], act[:,:,:2]), dim=2)
            )
    # calculate new act
    act = ~act & hum & f_act
    act2 = act
    #-------------------------
    # formation and extinction
    #-------------------------
    # update random values
    rnd_hum.random_(0, 10001)
    rnd_act.random_(0, 10001)
    rnd_ext.random_(0, 10001)
    # update cell states
    hum = hum_temp | (rnd_hum < P_hum)
    act = act | (rnd_act < P_act)
    cld = cld & (rnd_ext > P_ext)

    #--------------
    # Visualisation
    #--------------
    # flatten cld and create selection of all ones
    selection = cld.reshape(1, -1).squeeze() == 1

    ax3D.clear()
    ax3D.set_xlim(0,width)
    ax3D.set_ylim(0,depth)
    ax3D.set_zlim(0,height)
    plt.axis('off')
    ax3D.scatter(x[selection], y[selection], z[selection], s=1)
    fig.canvas.draw()
    fig.canvas.flush_events()
    